In [ ]:
*** Settings ***
Documentation    Get the calls history of a usage context (deleted items included)
...    response model:
...    - id	           : The technical ID of the call session,
...    - caller        : The caller phone number (or 'ANONYMOUS' in the case of a private call) in international format (ex. +33655887446),
...    - callee        : The callee phone number in international format (ex. +33655887446),
...    - date	       : [DEPRECATED] The start date of the call in  ISO-8601 format with timezone (ex. 2014-07-11T11:52:01.123Z)
...    - startDate     : The start date of the call in ISO-8601 format (ex. 2014-07-11T11:52:01.123Z),
...    - endDate       : The end date of the call in ISO-8601 format (ex. 2014-07-11T11:52:01.123Z),
...    - duration      : The duration between pickup and end of call (in seconds)
...    - direction     : The direction of the call, Enum = ["inbound", "outbound", "voicemailConsultation"],
...    - dndActivated  : Indicates if the 'do not disturb' mode was activated,
...    - read          : Reading flag (if the call has been seen or not)
...    test coverage (pairwise approach): working on both parametres and 
...    
...    *Test Design*:
...    | =tests=  |  =Authorization=  | =X-VNumbers-Zone-Id=  | =X-VNumbers-Service-Id=  | =X-OAPI-Application-Id=  | =voicemailid=  | =message ststus=  |
...    | =01=     |       Yes         |       YES             |       YES                |       YES                |       YES      |       NEW         |
...    | =02=     |       No          |       YES             |       YES                |       YES                |       YES      |       YES         |
...    | =03=     |      wrong        |       YES             |       YES                |       YES                |       YES      |       YES         |
...    | =04=     |       Yes         |       No              |       YES                |       YES                |       YES      |       YES         |
...    | =05=     |       Yes         |      wrong            |       YES                |       YES                |       YES      |       YES         |
...    | =06=     |       Yes         |       YES             |       No                 |       YES                |       YES      |       YES         |
...    | =07=     |       Yes         |       YES             |      wrong               |       YES                |       YES      |       YES         |
...    | =08=     |       Yes         |       YES             |       YES                |       No                 |       YES      |       YES         |
...    | =09=     |       Yes         |       YES             |       YES                |      wrong               |       YES      |       YES         |
...    | =10=     |       Yes         |       YES             |       YES                |       YES                |       No       |       YES         |
...    | =11=     |       Yes         |       YES             |       YES                |       YES                |       Wrong    |       YES         |

*** Variables ***
${first_boundary}    2029-07-15T00:00:00.0Z
${second_boundary}    2019-07-18T23:59:59.0Z
${Authorizationtoken}    Bearer ziWNfaMrCDnAribFf3VFGZoADjFJ
${ZoneID}       EUR
${serviceID}    e932c5c1-0716-45ec-b6b2-d17fa2a6be2c
${contextId}    cd6ec2f4-7efd-4152-b44f-efb8a4eddb48
${json_path}    ../AliasNumbersAutomation/data/tempdata/

*** Keywords ***
get time from timestamp
    [Arguments]    ${input}
    ${input}=    Convert To String    ${input}
    ${matches}=  Get Regexp Matches    ${input}    \\d{2}:\\d{2}:\\d{2}
    [return]    ${matches}[0]

count json array size
    [Documentation]    counts number of id elements in the json file
    [Arguments]    ${jsonfile}    
    ${full_json_data}=    Get file    ${jsonfile}
    ${json}=    evaluate    json.loads('''${full_json_data}''')    json
    ${ids}= 	Get Value From Json 	${json} 	$..id
    ${result}= 	Get Length    ${ids}
    [return]    ${result}     

calculate call duration
    [Arguments]    ${index}       ${file}
    ${history_data}=    Get file    ${file}
    ${json}=    evaluate    json.loads('''${history_data}''')    json
    ${duration}= 	Get Value From Json 	${json} 	[${index}].duration
    ${startDate}= 	Get Value From Json 	${json} 	[${index}].startDate
    ${endDate}= 	Get Value From Json 	${json} 	[${index}].endDate
    ${starttime}=    get time from timestamp    ${startDate}
    ${endtime}=    get time from timestamp    ${endDate}        
	${time} = 	Subtract Time From Time 	${endtime} 	${starttime}    exclude_millis=yes
	${time}= 	Convert To Integer    ${time} 	
    Should Be True 	${duration}[0] < ${time}
    #	Should Be Equal 	${time} 	${duration}[0]

all calls history request variable parameters
	[Arguments]  ${startdate} 	${enddate}    ${filename}    ${responsecode}
#    Create Session  orangedeveloper  ${base_url}    disable_warnings=1
    ${headers}=  Create Dictionary  Content-Type=application/json
    ...    Authorization=${Authorizationtoken}    X-VNumbers-Zone-Id=${ZoneID}    X-VNumbers-Service-Id=${serviceID}
    ${params} = 	Create Dictionary    startDate=${startdate}    endDate=${enddate}
    ${resp}=  Get Request  orangedeveloper    /contexts/${contextId}/calls/all?    headers=${headers}    params=${params},
    ${result}=    To Json    ${resp.text}     pretty_print=True
    ${json}=  Create File  ${json_path}/${filename}.json    ${result}   UTF-8     
    Should Be Equal As Strings    ${resp.status_code}    ${responsecode}

In [ ]:
*** Test Cases ***
check call history request - parameters tests
    [Documentation]    check response for parameters passed data
    [Template]    all calls history request variable parameters
    ${valid_parameters.startdate}      ${valid_parameters.enddate}      ${valid_parameters.filename}      ${valid_parameters.responsecode}        
    ${invalid_one_empty.startdate}    ${invalid_one_empty.enddate}    ${invalid_one_empty.filename}    ${invalid_one_empty.responsecode}        
    ${invalid_parameter_order.startdate}    ${invalid_parameter_order.enddate}    ${invalid_parameter_order.filename}    ${invalid_parameter_order.responsecode}        
    ${invalid_parameter_format_missing.startdate}    ${invalid_parameter_format_missing.enddate}    ${invalid_parameter_format_missing.filename}    ${invalid_parameter_format_missing.responsecode}        
    ${invalid_parameter_invalid_dates_future.startdate}    ${invalid_parameter_invalid_dates_future.enddate}    ${invalid_parameter_invalid_dates_future.filename}    ${invalid_parameter_invalid_dates_future.responsecode}        
    ${invalid_parameter_invalid_dates_past.startdate}      ${invalid_parameter_invalid_dates_past.enddate}      ${invalid_parameter_invalid_dates_past.filename}      ${invalid_parameter_invalid_dates_past.responsecode}        
